In [21]:
import arcpy

arcpy.env.overwriteOutput = True

In [22]:
import arcpy

def divide_polygon_with_cut_lines(input_polygon, output_feature_class):


    # Crear el cursor de búsqueda para obtener la geometría del polígono
    with arcpy.da.SearchCursor(input_polygon, ["SHAPE@"]) as cursor:

        for row in cursor:
            polygon = row[0]
            extent = polygon.extent

            # Crear una capa temporal de líneas
            cut_lines = "in_memory\\cut_lines"

            # Calcular las coordenadas de los puntos centrales
            center_x = (extent.XMin + extent.XMax) / 2
            center_y = (extent.YMin + extent.YMax) / 2

            # Crear las líneas de corte
            line1 = arcpy.Polyline(arcpy.Array([arcpy.Point(extent.XMin, center_y), arcpy.Point(extent.XMax, center_y)]))
            line2 = arcpy.Polyline(arcpy.Array([arcpy.Point(center_x, extent.YMin), arcpy.Point(center_x, extent.YMax)]))

            # Crear la capa de líneas de corte temporal
            arcpy.management.CreateFeatureclass("in_memory", "cut_lines", "POLYLINE", spatial_reference=input_polygon)

            # Insertar las líneas en la capa de líneas de corte
            with arcpy.da.InsertCursor(cut_lines, ["SHAPE@"]) as insert_cursor:
                insert_cursor.insertRow([line1])
                insert_cursor.insertRow([line2])

            # Dividir el polígono utilizando las líneas de corte
            arcpy.management.FeatureToPolygon([input_polygon, cut_lines], output_feature_class)

            # Eliminar la capa temporal de líneas
            arcpy.management.Delete(cut_lines)


In [23]:
arcpy.env.workspace = r"C:\docsProyectos\8.GESTVALT\1.GIS\PRO\GESVALT\GESVALT.gdb"

FC_SECTOR = r"C:\docsProyectos\8.GESTVALT\1.GIS\PRO\GESVALT\GESVALT.gdb\sector"

divide_polygon_with_cut_lines(FC_SECTOR,'Corte_Prueba')